# Direct Optimization of initial weights via Linear Programming

For a fully connected feed forward neural net using RELU activation functions $f$, for a given input $x \in D$ and parameter set $P = (W_1, W_2, ...)$ where each $W_i$ is the weight matrix for a layer of the network, then there exists matrix $A$ such that $f(x, P) = Ax$.

Consider the following generalization loss function, defined as:

$L_{gen} = (f(x, P) - f(x, P^*)) - (f(x^*, P) - f(x^*, P^*))$

where $x^* = x + x'$ and $P^* = P + P'$ for some small perturbations $x', P'$ (applying a different perturbation to each weight matrix in $P'$)

Then if perturbations $x', P'$ do not change the activation pattern of $f(x, P)$, $L_{gen}$ is equivalent to:

$L_{gen} = Ax - (A + A')x - A(x+x') + (A+A')(x+x')$

$ = Ax - Ax - A'x - Ax -Ax' + Ax + Ax' + A'x + A'x'$

$ = A'x'$

Therefore, we can obtain a small value of $L_{gen}$ by finding an initialization $P$ which has few activation patterns for the training data. Since the activation function is RELU, this is equivalent to reducing the amount of times RELU sets an element to 0.